# importing data

In [1]:
"""
Simple tester for the deep3d
"""

import tensorflow as tf
import Deep3D as deep3d
import utils
import numpy as np
import os
import os.path
import h5py
import matplotlib as plt

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
inria_file = '/a/data/deep3d_data/inria_data.h5'
h5f = h5py.File(inria_file,'r')

X_train = h5f['X_0'][:400,10:170,16:304,:]
Y_train = h5f['Y_0'][:400,10:170,16:304,:]
X_val = h5f['X_0'][400:,10:170,16:304,:]
Y_val = h5f['Y_0'][400:,10:170,16:304,:]
   
    
h5f.close()

In [ ]:
num_batches = 10
batchsize = 50
device1 = '/gpu:0'
#device1 = '/cpu:0'

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

with tf.device(device1):
    sess = tf.Session(config=config)

    images = tf.placeholder(tf.float32, [batchsize, 160, 288, 3])
    true_out = tf.placeholder(tf.float32, [batchsize, 160, 288, 3])
    
    train_mode = tf.placeholder(tf.bool)

    net = deep3d.Deep3Dnet('./vgg19.npy')
    net.build(images, train_mode)

    #print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print 'var count:'
    print(net.get_var_count())
        
    sess.run(tf.global_variables_initializer())
    
    # simple 1-step training
    cost = tf.reduce_sum((net.prob - true_out) ** 2)
    train = tf.train.GradientDescentOptimizer(0.002).minimize(cost)
    print "start training"
    for i in xrange(num_batches):
        image_mask = np.random.choice(X_train.shape[0],batchsize)
        images_in = X_train[image_mask,:,:,:]
        labels_in = Y_train[image_mask,:,:,:]
        _, cost_val = sess.run([train,cost], feed_dict={images: images_in, true_out: labels_in, train_mode: True})
        print 'cost:' + str(cost_val)
       

var count:
138408071
start training
